# Update fits header and move fits files

## import modules

우선 필요한 모듈을 import합니다.

In [1]:
from pathlib import Path
import shutil
import os
import ysfitsutilpy as yfu

import _Python_utilities
import _astro_utilities

import warnings
warnings.filterwarnings('ignore')

## 디렉토리 환경 설정

새로 촬영한 파일들은 아래 폴더에 복사를 합니다. 처리해야 할 파일들이 들어 있는 폴더를 목록으로 만들어 둡니다.
이때 pathlib을 사용하면 운영체제에 관게 없이 파일 경로를 객체로 만들어 쉽게 다룰 수 있게 됩니다. 

In [2]:
#%%
verbose = False  # True     
Owrite = False   
#######################################################
# Set directory variables.
#######################################################

BASEDIR = Path(r'O:\\')   # for windows
BASEDIR = Path(r"/mnt/Rdata/ASTRO_data")  # for ubuntu
# BASEDIR = Path("/Volumes/OBS_data")  # for mac OS
 
DOINGDIR = BASEDIR/ _astro_utilities.CCD_NEW_dir

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(DOINGDIR))
if verbose == True :
    print ("DOINGDIRs: ", format(DOINGDIRs))
    print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))
#######################################################    
 

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin/']
len(DOINGDIRs):  1


## 파일 이름 변수 설정

파일명을 통해 장비 및 촬영 정보를 확인하기 위하여 폴더명/파일명은 다음과 같은 규칙으로 저장하고 있습니다. 

> 폴더명(파일명) 구조: [object name]_[image type]_[filter]_[OBS datetime]_[exptime]_[optic name]_[detector name]_[OBS temperature]_[binning].fit

fits header에서는 다음의 keyword를 활용합니다. 

In [3]:
print("_astro_utilities.fnameKEYs :", _astro_utilities.fnameKEYs)

_astro_utilities.fnameKEYs : ['OBJECT', 'IMAGETYP', 'FILTER', 'DATE-OBS', 'EXPOSURE', 'OPTIC', 'CCDNAME', 'CCD-TEMP', 'XBINNING']


## fits header update and move to
``DOINGDIRs``에 list 형태로 들어있는 디렉토리 경로안에 모든 파일들을 ``yfu.make_summary`` 함수를 이용하여 dataframe으로 만들어 파일명 하나하나 fits 헤더를 업데이트 합니다.

In [4]:
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    if verbose == True : 
        print("DOINGDIR", DOINGDIR)
        print(f"Starting: {str(DOINGDIR.parts[-1])}")
    # NEWUPDIR = DOINGDIR.parents[1] /_astro_utilities.CCD_NEWUP_dir
    try:
        summary = yfu.make_summary(DOINGDIR/"*.fit*",
                                    verify_fix=True,
                                    ignore_missing_simple=True,
                                    )
        if summary is not None : 

            if verbose == True : 
                print("summary: ", summary)
                print("len(summary)", len(summary))

            for _, row in summary.iterrows():

                if verbose == True : 
                    print (row["file"])   # 파일명 출력
                fpath = Path(row["file"])

                hdul = _astro_utilities.KevinFitsUpdater(fpath,
                                                # checkKEYs = ["OBJECT", "TELESCOP", "OPTIC", "CCDNAME", 'FILTER',
                                                #             #"GAIN", "EGAIN", "RDNOISE", 
                                                #             "PIXSCALE", "FOCALLEN", "APATURE", "CCD-TEMP",
                                                #             'XPIXSZ', 'YPIXSZ',
                                                #             "XBINNING", "YBINNING", "FLIPSTAT", "EXPTIME", "EXPOSURE"],
                                                # imgtype_update=True,
                                                # fil_update=False,
                                                verbose = verbose, 
                                                )
                if verbose == True :
                    print("hdul: ", hdul)

                new_fname = ""
                suffix = ".fit"

                for KEY in _astro_utilities.fnameKEYs :
                    if KEY in ["OBJECT", "IMAGETYP", "FILTER", 
                                "OPTIC", "CCDNAME"] :
                        new_fname += str(row[KEY])+"_"
                    
                    if KEY == "DATE-OBS" : 
                        new_fname += row[KEY][:19].replace("T","-").replace(":","-")+"_"

                    if KEY == "EXPOSURE" : 
                        new_fname += str(int(row[KEY]))+"sec_"

                    if KEY == "CCD-TEMP" : 
                        try:
                            new_fname += str(int(row[KEY]))+"c_"
                        except:
                            new_fname += (row[KEY])+"c_"
                    if KEY == "XBINNING" : 
                        new_fname += str(row[KEY])+"bin"+suffix
                if verbose == True :
                    print(new_fname)                      
                new_folder = _astro_utilities.get_new_foldername_from_filename(new_fname)
                if verbose == True :
                    print("new_folder: ", new_folder)
                new_fpath =  BASEDIR /_astro_utilities.A3_CCD_obs_raw_dir / new_folder / new_fname
                if verbose == True :
                    print("new_fpath: ", new_fpath)

                if not new_fpath.parents[0].exists():
                    os.makedirs(f'{new_fpath.parents[0]}')
                    if verbose == True :
                        print(f'{new_fpath.parts[-2]} is created')  
            
                if new_fpath.exists() :
                    if verbose == True :
                        print(f'{new_fpath} is already exist')
                    duplicate_fpath = BASEDIR / _astro_utilities.CCD_duplicate_dir / new_fpath.name
                    if Owrite == False:
                        shutil.move(fpath, duplicate_fpath)
                        if verbose == True :
                            print(f'{fpath.parts[-1]} is move to duplicate folder...')
                    else :
                        shutil.move(str(fpath), str(new_fpath))
                        if verbose == True :
                            print(f"move {str(fpath.name)} to {str(new_fpath)}")
                else : 
                    shutil.move(str(fpath), str(new_fpath))
                    if verbose == True :
                        print(f"move {str(fpath.name)} to {str(new_fpath)}")

    except Exception as err :
        print("X"*60)
        print(err)
        pass

DOINGDIR /mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin
Starting: A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin
All 54 keywords (guessed from /mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin/A924UB-240827-12_LIGHT__2024-08-27_20-10-09_30.00_OON300_ASCOM ToupTek Camera Driver (ASCOM)_9.70_1x1.fits) will be loaded.
summary:                                                  file  filesize  SIMPLE  \
0  /mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-...  51624000    True   
1  /mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-...  51624000    True   
2  /mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-...  51624000    True   

   BITPIX  NAXIS  NAXIS1  NAXIS2  EXTEND  BZERO IMAGETYP  ...  FOCUSPOS  \
0      16      2    6216    4152    True  32768    LIGHT  ...      2140   
1      16      2    62

빈 디텍토리를 지우는 코드입니다.

In [5]:
#%%   
#############################################################################
#Check and delete empty folder....
#############################################################################
for i in range(4):
    DOINGDIR = ( BASEDIR/ _astro_utilities.CCD_NEW_dir)         
    DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(DOINGDIR))
    if verbose == True :
        print ("DOINGDIRs: ", format(DOINGDIRs))
        print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

    for DOINGDIR in DOINGDIRs :    
        if len(os.listdir(str(DOINGDIR))) == 0 :
            shutil.rmtree(f"{str(DOINGDIR)}") # Delete..
            if verbose == True :
                print (f"rmtree {str(DOINGDIR)}")
        else : 
            fpaths = _Python_utilities.getFullnameListOfallFiles(str(DOINGDIR))
            if verbose == True :
                print("fpaths", fpaths)

            for fpath in fpaths[:]:
                if verbose == True :
                    print("fpath", fpath)

                if fpath[-4:].lower() in [".txt", "xisf", ".zip", ".png", ".log",
                                            "seal", "tiff", ".axy", "atch", "lved",
                                            "rdls", "xyls", "corr", "xosm", ".ini",
                                            ".wcs", ".csv"] \
                                        and os.path.isfile(fpath):
                    os.remove("{}".format(fpath))
                    if verbose == True :
                        print("{} is removed...".format(fpath)) 


DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin/']
len(DOINGDIRs):  1
fpaths ['/mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin/A924UB-240827-12_LIGHT__2024-08-27_20-10-09_30.00_OON300_ASCOM ToupTek Camera Driver (ASCOM)_9.70_1x1.fits', '/mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin/A924UB-240827-12_LIGHT__2024-08-27_20-11-25_60.00_OON300_ASCOM ToupTek Camera Driver (ASCOM)_9.40_1x1.fits', '/mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera-Driver-(ASCOM)_-_1x1bin/A924UB-240827-12_LIGHT__2024-08-27_20-13-01_90.00_OON300_ASCOM ToupTek Camera Driver (ASCOM)_9.70_1x1.fits']
fpath /mnt/Rdata/ASTRO_data/A1_CCD_new_files/A924UB-240827-12-LIGHT_-_2024-08-27_-_OON300_ASCOM-ToupTek-Camera